Modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/code_agents.ipynb

Selecting a Playlist for the Party Using smolagents

In [1]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_music_recommendations = web_search(query="best party music recommendations")                               
  wayne_mansion_music = web_search(query="proposed music for Wayne's mansion party")                               
  print("Party music recommendations:", party_music_recommendations)                                               
  print("Music for Wayne's mansion party:", wayne_mansion_music)                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Party music recommendations: ## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

[50+ Best Songs For Your Next Party in 2025 - Aleka's 
Get-Together](https://alekasgettogether.com/top-songs-for-any-party/)
Whether you're planning a How To Host The Best Party At Home or just want to liven up your playlist, we've got you 
covered with 100 crowd-pleasing tunes. From timeless classics to modern hits, these songs are guaranteed to keep 
the energy high and the good times rolling all night long. 50 Popular Party Songs. Uptown Funk - Mark Ronson ft ...

[Best Songs To Party: DJ's Ultimate Party Songs Playlist - 
Top40Weekly.com](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist. If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference. From classic hits 
to ...

[The Top 100 Best Party Songs of All Time - 
LiveAbout](https://www.liveabout.com/top-best-party-songs-of-all-time-3248355)
"Macarena" then spent 14 weeks at No. 1 on the U.S. pop singles chart. For more than a year this was one of the 
most popular special event songs in the United States. It still works well as a charming party song encouraging 
everyone to join in on the simple dance.

[40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
The 40 best party songs ever made. It's party time guys - don your dancing shoes and have your speaker at the 
ready. Thursday 13 February 2025. Share. Copy Link. Facebook Twitter Pinterest ...

[The 101 Best Party Songs Of All Time - Music Grotto](https://www.musicgrotto.com/best-party-songs/)
A pounding beat and dramatic vocals make this song one of the best party songs to come out of Europe, and the 
song's music video features a strange and compelling premise of a dead man being reanimated and falling in love 
with a woman. Tjis Verwest, (a.k.a. Tiësto) composed the song with help from James Bell, Julia Karlsson, and Anton 
Rundberg.

[Best Party Playlist: 25 High-Energy Songs for the Ultimate 
Celebration](https://audioassemble.com/best-party-playlist/)
Whether you're planning a weekend house party, a birthday bash, or an impromptu dance-off, every celebration 
deserves the perfect soundtrack. Our selection of 25 high-energy songs are tried-and-true crowd pleasers that 
seamlessly blend different genres and eras, ensuring there's something for everyone.

[Best Party Songs — Best Dance Music - 
Cosmopolitan](https://www.cosmopolitan.com/uk/entertainment/a46785812/best-songs-for-party-playlist/)
3. 'Taki Taki,' by DJ Snake, feat. Selena Gomez, Ozuna, and Cardi B. If both Selena Gomez and Cardi B are featured 
on a track, it just has to be a good one.This powerful Latinx collab was made ...

[The Best Party Songs of All Time (Our Playlists)](https://www.ispytunes.com/post/best-party-songs)
Choosing the best party songs can turn your event into an unforgettable experience. Whether you're looking for 
timeless classics or the latest hits, a well-crafted playlist sets the mood and keeps the energy high. From disco 
beats to pop anthems and hip-hop jams, these tracks bring people together on the dance floor. ...

[100 Best Party Songs: The Ultimate Party Playlist](https://audiocaptain.com/best-party-songs/)
Best 2000s Party Songs. Stylistically speaking, music that dominated the charts in the 2000s was nondescript.. 
There was not much new regarding musical directions. C

[Step 0: Duration 9.01 seconds| Input tokens: 2,085 | Output tokens: 124]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(recommended_songs)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['September - Earth, Wind & Fire', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'Macarena - Los 
Del Rio', 'Taki Taki - DJ Snake, Selena Gomez, Ozuna, Cardi B', 'Mansion Party - Tee Grizzley, Fredo Bang', "VIP - 
Amanda La'Depeche, David Guetta", 'Roar - Katy Perry', 'Party in the USA - Miley Cyrus', 'Shake It Off - Taylor 
Swift', "Can't Stop the Feeling! - Justin Timberlake"]

[Step 2: Duration 5.95 seconds| Input tokens: 11,512 | Output tokens: 483]

['September - Earth, Wind & Fire',
 'Uptown Funk - Mark Ronson ft. Bruno Mars',
 'Macarena - Los Del Rio',
 'Taki Taki - DJ Snake, Selena Gomez, Ozuna, Cardi B',
 'Mansion Party - Tee Grizzley, Fredo Bang',
 "VIP - Amanda La'Depeche, David Guetta",
 'Roar - Katy Perry',
 'Party in the USA - Miley Cyrus',
 'Shake It Off - Taylor Swift',
 "Can't Stop the Feeling! - Justin Timberlake"]

Using Python Imports Inside the Agent


In [ ]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━